In [143]:
import pyodbc
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
import random
import numpy as np
import tensorflow as tf

server = 'cyse492group8.database.windows.net'
database = 'CYSE492_Project'
username = 'cyseadmin'
password = 'GroupProject492'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = conn.cursor()

In [144]:
cursor.execute("SELECT [Control Identifier],[Control Text]  FROM [dbo].['NIST_800.53_controls$']")

labels = []
texts = []
for row in cursor:
    texts.append(row[1])
    if 'AC' in row[0]:
        labels.append(1)
    if 'AT' in row[0]:
        labels.append(2)
    if 'AU' in row[0]:
        labels.append(3)
    if 'CA' in row[0]:
        labels.append(4)
    if 'CM' in row[0]:
        labels.append(5)
    if 'CP' in row[0]:
        labels.append(6)
    if 'IA' in row[0]:
        labels.append(7)
    if 'IR' in row[0]:
        labels.append(8)
    if 'MA' in row[0]:
        labels.append(9)
    if 'MP' in row[0]:
        labels.append(10)
    if 'PE' in row[0]:
        labels.append(11)
    if 'PL' in row[0]:
        labels.append(12)
    if 'PM' in row[0]:
        labels.append(13)
    if 'PS' in row[0]:
        labels.append(14)
    if 'PT' in row[0]:
        labels.append(15)
    if 'RA' in row[0]:
        labels.append(16)
    if 'SA' in row[0]:
        labels.append(17)
    if 'SC' in row[0]:
        labels.append(18)
    if 'SI' in row[0]:
        labels.append(19)
    if 'SR' in row[0]:
        labels.append(20)

x_test = []
y_test = []



for i in range(115):
    random_index = random.randrange(len(labels))
    x_test.append(texts.pop(random_index))
    y_test.append(labels.pop(random_index))

In [145]:
# Hyperparameters
max_words = 15000 # max number of words to use in the vocabulary
max_len = 250 # max length of each text (in terms of number of words)
embedding_dim = 100 # dimension of word embeddings
lstm_units = 64 # number of units in the LSTM layer
num_classes = len(set(labels)) + 1 # number of classes
#num_classes2 = len(set(y_test)) + 1

# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad the sequences so they all have the same length
x = pad_sequences(sequences, maxlen=max_len)


# Create one-hot encoded labels
y = keras.utils.to_categorical(labels, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Build the model
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(lstm_units))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x, y, batch_size=32, epochs=20)

x_test = list(x_test)
# Tokenize the texts and create a vocabulary
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_test)
sequences = tokenizer.texts_to_sequences(x_test)


# Pad the sequences so they all have the same length
x_test = pad_sequences(sequences, maxlen=max_len)

#Test the model
accr = model.evaluate(x_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

#for i in range(10):
#    result = tf.argmax(model.predict(tf.expand_dims(texts[i], 0)), axis=1)
#    print(result.numpy(), labels[i])

Epoch 1/20
34/34 [==============================] - 7s 141ms/step - loss: 2.8874 - accuracy: 0.1220
Epoch 2/20
34/34 [==============================] - 5s 134ms/step - loss: 2.7039 - accuracy: 0.1555
Epoch 3/20
34/34 [==============================] - 5s 132ms/step - loss: 2.5378 - accuracy: 0.2179
Epoch 4/20
34/34 [==============================] - 4s 131ms/step - loss: 2.2460 - accuracy: 0.2905
Epoch 5/20
34/34 [==============================] - 5s 134ms/step - loss: 1.9510 - accuracy: 0.4488
Epoch 6/20
34/34 [==============================] - 4s 132ms/step - loss: 1.7132 - accuracy: 0.5298
Epoch 7/20
34/34 [==============================] - 4s 131ms/step - loss: 1.4612 - accuracy: 0.5894
Epoch 8/20
34/34 [==============================] - 5s 134ms/step - loss: 1.2675 - accuracy: 0.6750
Epoch 9/20
34/34 [==============================] - 4s 131ms/step - loss: 1.1325 - accuracy: 0.7207
Epoch 10/20
34/34 [==============================] - 5s 133ms/step - loss: 0.9836 - accuracy: 0.7439

In [146]:
print(x_test.shape)
print(y_test.shape)
print(x.shape)
print(y.shape)
print(y_test)
print(y)

(115, 250)
(115, 21)
(1074, 250)
(1074, 21)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
